#  A simple deep CNN
[reference](https://keras.io/examples/cifar10_cnn/)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D, Convolution1D
import os

In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style('whitegrid')
sns.set_palette('Set2')
# %matplotlib inline
plt.rcParams.update({'figure.titlesize': 'larger', 'legend.fontsize': 15.0})

In [3]:
batch_size = 32
num_classes = 2
epochs = 10
data_augmentation = True
# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_trained_model_one_year.h5'

In [4]:
data =pd.read_csv('data_pre_processed.csv')

In [5]:
print('Total data shape is ', data.shape)
data.head()

Total data shape is  (57996, 112)


,first_careunit,last_careunit,age,age_category,gender,marital_status,insurance,urea_n_min,urea_n_max,urea_n_mean,...,spo2_max_mv,spo2_mean_mv,vent_mv,rrt_mv,urineoutput_mv,oasis_mv,lods_mv,sirs_mv,thirty_days,one_year
0,1.062189,1.042783,0.020618,7.0,0.889068,0.801375,0.722122,-0.110132,0.345616,0.042146,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.338382,0.334179,-0.175699,6.0,-1.124773,0.801375,0.722122,-0.035009,1.564791,1.406861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.062189,1.042783,-0.623562,4.0,0.889068,-2.394261,-1.738451,-0.035009,-0.296056,-0.244412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,-0.338382,0.334179,-0.056425,7.0,0.889068,0.801375,0.722122,0.866469,-0.039387,0.257946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,-1.038668,-0.374426,-0.659244,3.0,-1.124773,-2.394261,-1.738451,-0.260378,-0.648975,-0.588887,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
columns = list(data.columns)
X = data[columns[0:-2]].astype(np.float32) # fetures
# y = data[columns[-2]].astype(np.float32)  # 30 days 
y = data[columns[-1]].astype(np.float32)  # one year

from sklearn.model_selection import train_test_split
# The data, split between train and test sets:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (38857, 110)
38857 train samples
19139 test samples


In [7]:
model = Sequential()
model.add(Conv1D(32, 3, padding='same', input_shape=[X_train.shape[1],1]))
model.add(Activation('relu'))
model.add(Conv1D(32, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv1D(64, 3, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# initiate RMSprop optimizer
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

X_train = X_train.values[..., None].astype('float32')
X_test = X_test.values[..., None].astype('float32')
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 110, 32)           128       
_________________________________________________________________
activation (Activation)      (None, 110, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 108, 32)           3104      
_________________________________________________________________
activation_1 (Activation)    (None, 108, 32)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 54, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 54, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 54, 64)            6208      
__________

In [9]:
X_train.shape

(38857, 110, 1)

In [10]:
hist = model.fit(X_train, y_train,
          batch_size=64,
          epochs=15,
          verbose = 1,
          validation_data=(X_test, y_test),
          shuffle=True,
            callbacks= [keras.callbacks.EarlyStopping()] )


Train on 38857 samples, validate on 19139 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
38857/38857 [==============================] - 23s 589us/sample - loss: 0.4723 - acc: 0.7777 - val_loss: 0.4135 - val_acc: 0.8069
Epoch 2/15
38857/38857 [==============================] - 23s 581us/sample - loss: 0.4032 - acc: 0.8139 - val_loss: 0.3599 - val_acc: 0.8366
Epoch 3/15
38857/38857 [==============================] - 24s 613us/sample - loss: 0.3781 - acc: 0.8256 - val_loss: 0.3431 - val_acc: 0.8435
Epoch 4/15
38857/38857 [==============================] - 25s 652us/sample - loss: 0.3593 - acc: 0.8354 - val_loss: 0.3359 - val_acc: 0.8489
Epoch 5/15
38857/38857 [==============================] - 25s 649us/sample - loss: 0.3522 - acc: 0.8398 - val_loss: 0.3247 - val_acc: 0.8483
Epoch 6/15
38857/38857 [==============================] - 25s 653us/sample - loss: 0.3392 - acc: 0.8458 - val_loss: 0.3104 - val_acc: 0.8616
Epoch 7/15
38857/38857 [==============================] 

In [12]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\64877\Repositories\DS590 Final Project\DS_final\Process Books\saved_models\keras_trained_model_one_year.h5 
19139/19139 [==============================] - 3s 166us/sample - loss: 0.2662 - acc: 0.8853
Test loss: 0.2662147941606297
Test accuracy: 0.88531274


# Modified VGG16
[reference](https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3)

In [13]:
model = Sequential()
model.add(ZeroPadding1D(1,input_shape=[X_train.shape[1],1]))
model.add(Convolution1D(64, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(ZeroPadding1D(1))
model.add(Convolution1D(128, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(ZeroPadding1D(1))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(ZeroPadding1D(1))
model.add(Convolution1D(512, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(512, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(512, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))



In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding1d (ZeroPadding1 (None, 112, 1)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 110, 64)           256       
_________________________________________________________________
zero_padding1d_1 (ZeroPaddin (None, 112, 64)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 110, 64)           12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 55, 64)            0         
_________________________________________________________________
zero_padding1d_2 (ZeroPaddin (None, 57, 64)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 55, 128)           24704     
__________

In [ ]:
# load weights 
# filepath = './saved_models/Modified_VGG_one_year.h5'
# model.load_weights(filepath, by_name=False)

In [16]:
hist = model.fit(X_train, y_train,
              batch_size=512,
              epochs=10,
              validation_data=(X_test, y_test),
              shuffle=True,
                callbacks = [keras.callbacks.EarlyStopping()])

Train on 38857 samples, validate on 19139 samples
Epoch 1/5
38857/38857 [==============================] - 256s 7ms/sample - loss: 0.4368 - acc: 0.7963 - val_loss: 0.4131 - val_acc: 0.8156
Epoch 2/5
38857/38857 [==============================] - 240s 6ms/sample - loss: 0.3781 - acc: 0.8308 - val_loss: 0.3539 - val_acc: 0.8369
Epoch 3/5
38857/38857 [==============================] - 244s 6ms/sample - loss: 0.3301 - acc: 0.8557 - val_loss: 0.3301 - val_acc: 0.8580
Epoch 4/5
38857/38857 [==============================] - 255s 7ms/sample - loss: 0.3068 - acc: 0.8674 - val_loss: 0.3200 - val_acc: 0.8645
Epoch 5/5
38857/38857 [==============================] - 258s 7ms/sample - loss: 0.2904 - acc: 0.8771 - val_loss: 0.3056 - val_acc: 0.8712


In [17]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, 'Modified_VGG_one_year.h5')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\64877\Repositories\DS590 Final Project\DS_final\Process Books\saved_models\Modified_VGG_one_year.h5 
19139/19139 [==============================] - 26s 1ms/sample - loss: 0.3056 - acc: 0.8712
Test loss: 0.30559010178561563
Test accuracy: 0.8712054
